## Libraries

In [1]:
import argparse
import math
import os
import numpy as np
import torch as T
import torch.nn.functional as F
from tqdm import tqdm
from utility.VAE_Text_Generation.helper_functions import get_cuda
from utility.VAE_Text_Generation.model import VAE

import torch
print('Cuda available:', torch.cuda.is_available())
try: 
    cuda_id = torch.cuda.current_device()
    print('Cuda_id: ', cuda_id)
    print(torch.cuda.get_device_name(cuda_id))
except:
    print('No Cuda :(')

Cuda available: True
Cuda_id:  0
NVIDIA GeForce RTX 4060 Laptop GPU


## Opt ARGS Class

In [2]:
class Args:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

# Define the dictionary with default values
args_dict = {
    'batch_size': 8,
    'n_vocab': 12000,
    'epochs': 1000,
    'n_hidden_G': 512,
    'n_layers_G': 2,
    'n_hidden_E': 512,
    'n_layers_E': 1,
    'n_z': 100,
    'word_dropout': 0.5,
    'rec_coef': 7,
    'lr': 0.00001,
    'gpu': 0,
    'n_highway_layers': 2,
    'n_embed': 300,
    'out_num': 30000,
    'unk_token': "<unk>",
    'pad_token': "<pad>",
    'start_token': "<sos>",
    'end_token': "<eos>",
    'dataset': "gab",
    'training': True,
    'resume_training': False
}

opt = Args(**args_dict)

In [3]:
# Training Parameters
opt.dataset = 'conansp' 
print('opt.dataset', opt.dataset)
opt.training = True 
print('opt.training', opt.training)
opt.resume_training = False
print('opt.resume_training', opt.resume_training)
opt.n_embed = 300
print('opt.n_embed', opt.n_embed)
opt.epochs = 1000
print('opt.epochs', opt.epochs)


opt.dataset conansp
opt.training True
opt.resume_training False
opt.n_embed 300
opt.epochs 1000


In [4]:
save_path = "tmp/saved_VAE_models/" + opt.dataset + ".tar"
print(save_path)

if not os.path.exists("tmp/saved_VAE_models"):
    os.makedirs("tmp/saved_VAE_models")

os.environ["CUDA_VISIBLE_DEVICES"] = str(opt.gpu)

candidates_path = opt.dataset + '_for_VAE.txt'

tmp/saved_VAE_models/conansp.tar


## VAE Iterators and GLOVE Download

Usar Coneccion a internet

In [5]:
from utility.VAE_Text_Generation.dataset_SP import get_iterators
train_iter, val_iter, vocab = get_iterators(opt, path='./data/', fname=candidates_path)
opt.n_vocab = len(vocab)

/home/jaimelicea/miniconda3/envs/gps/lib/python3.9/site-packages/torchtext/vocab.py:432: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.itos, self.stoi, self.vectors, se

In [6]:
print(list(vocab.stoi.items())[:20])
print(vocab.vectors.numpy().shape)

[('<unk>', 0), ('<pad>', 1), ('<sos>', 2), ('<eos>', 3), ('de', 4), ('que', 5), ('la', 6), ('no', 7), ('los', 8), ('y', 9), ('a', 10), ('es', 11), ('el', 12), ('en', 13), ('las', 14), ('una', 15), ('un', 16), ('por', 17), ('son', 18), ('se', 19)]
(10694, 300)


In [7]:
if opt.training:
    vae = VAE(opt)
    vae.embedding.weight.data.copy_(vocab.vectors)
    vae = get_cuda(vae)
    trainer_vae = T.optim.Adam(vae.parameters(), lr=opt.lr)
else:
    checkpoint = T.load(save_path)
    vae = checkpoint['vae_dict']
    trainer_vae = checkpoint['vae_trainer']
    if 'opt' in checkpoint:
        opt_old = checkpoint['opt']
        print(opt_old)

## Training and Generation

In [8]:
def create_generator_input(x, train):
    G_inp = x[:, 0:x.size(1)-1].clone()	                    #input for generator should exclude last word of sequence
    if train == False:
        return G_inp
    r = np.random.rand(G_inp.size(0), G_inp.size(1))        #Perform word_dropout according to random values (r) generated for each word
    for i in range(len(G_inp)):
        for j in range(1, G_inp.size(1)):
            if r[i, j] < opt.word_dropout and G_inp[i, j] not in [vocab.stoi[opt.pad_token], vocab.stoi[opt.end_token]]:
                G_inp[i, j] = vocab.stoi[opt.unk_token]
    return G_inp



def train_batch(x, G_inp, step, train=True):
    logit, _, kld = vae(x, G_inp, None, None)
    logit = logit.view(-1, opt.n_vocab)	                    #converting into shape (batch_size*(n_seq-1), n_vocab) to facilitate performing F.cross_entropy()
    x = x[:, 1:x.size(1)]	                                #target for generator should exclude first word of sequence
    x = x.contiguous().view(-1)	                            #converting into shape (batch_size*(n_seq-1),1) to facilitate performing F.cross_entropy()
    rec_loss = F.cross_entropy(logit, x)
    kld_coef = (math.tanh((step - 15000)/1000) + 1) / 2
    # kld_coef = min(1,step/(200000.0))
    loss = opt.rec_coef*rec_loss + kld_coef*kld
    if train==True:	                                    #skip below step if we are performing validation
        trainer_vae.zero_grad()
        loss.backward()
        trainer_vae.step()
    return rec_loss.item(), kld.item()

def training():
    start_epoch = step = 0
    if opt.resume_training:
        step, start_epoch = checkpoint['step'], checkpoint['epoch']
    for epoch in range(start_epoch, opt.epochs):
        vae.train()
        train_rec_loss = []
        train_kl_loss = []
        for batch in train_iter:
            x = get_cuda(batch.text) 	                                #Used as encoder input as well as target output for generator
            G_inp = create_generator_input(x, train=True)
            rec_loss, kl_loss = train_batch(x, G_inp, step, train=True)
            train_rec_loss.append(rec_loss)
            train_kl_loss.append(kl_loss)
            step += 1

        vae.eval()
        valid_rec_loss = []
        valid_kl_loss = []
        for batch in val_iter:
            x = get_cuda(batch.text)
            G_inp = create_generator_input(x, train=False)
            with T.autograd.no_grad():
                rec_loss, kl_loss = train_batch(x, G_inp, step, train=False)
            valid_rec_loss.append(rec_loss)
            valid_kl_loss.append(kl_loss)

        train_rec_loss = np.mean(train_rec_loss)
        train_kl_loss = np.mean(train_kl_loss)
        valid_rec_loss = np.mean(valid_rec_loss)
        valid_kl_loss = np.mean(valid_kl_loss)

        print("No.", epoch, "T_rec:", '%.2f' % train_rec_loss, "T_kld:", '%.2f' % train_kl_loss, "V_rec:", '%.2f' % valid_rec_loss, "V_kld:", '%.2f' % valid_kl_loss)
        if epoch >= 50 and epoch % 10 == 0:
            print('save model ' + str(epoch) + '...')
            T.save({'epoch': epoch + 1, 'vae_dict': vae, 'vae_trainer': trainer_vae, 'step': step, 'opt': opt}, save_path)
            generate_sentences(5)
 
def generate_sentences(n_examples, save=0):
    vae.eval()
    out = []
    for i in tqdm(range(n_examples)):
        z = get_cuda(T.randn([1, vae.n_z]))
        h_0 = get_cuda(T.zeros(vae.generator.n_layers_G, 1, vae.generator.n_hidden_G))
        c_0 = get_cuda(T.zeros(vae.generator.n_layers_G, 1, vae.generator.n_hidden_G))
        G_hidden = (h_0, c_0)
        G_inp = T.LongTensor(1, 1).fill_(vocab.stoi[opt.start_token])
        G_inp = get_cuda(G_inp)
        out_str = ""
        while (G_inp[0][0].item() != vocab.stoi[opt.end_token]) and (G_inp[0][0].item() != vocab.stoi[opt.pad_token]):
            with T.autograd.no_grad():
                logit, G_hidden, _ = vae(None, G_inp, z, G_hidden)
            probs = F.softmax(logit[0], dim=1)
            G_inp = T.multinomial(probs, 1)
            out_str += (vocab.itos[G_inp[0][0].item()]+" ")
        # print(out_str[:-6])
        out.append(out_str[:-6])
    if save:
        original = []
        with open('./data/' + candidates_path, 'r') as fin:
            for line in fin:
                original.append(line.strip())
        fname = './data/' + opt.dataset + '_candidates.txt'
        with open(fname, 'w') as fout:
            for i in out + original:
                fout.write(i)
                fout.write('\n')

Esto guarda un txt de candidatos de opt.dataset en: './data/opt.dataset_candidates.txt'


In [ ]:
if opt.training or opt.resume_training:
    training()
    generate_sentences(opt.out_num, save=1)
else:
    generate_sentences(opt.out_num, save=1)

No. 0 T_rec: 7.20 T_kld: 294.04 V_rec: 7.73 V_kld: 597.01
No. 1 T_rec: 4.68 T_kld: 942.82 V_rec: 6.89 V_kld: 1013.16
No. 2 T_rec: 4.47 T_kld: 1203.00 V_rec: 6.81 V_kld: 1179.70
No. 3 T_rec: 4.42 T_kld: 1330.41 V_rec: 6.72 V_kld: 1266.23
No. 4 T_rec: 4.29 T_kld: 1217.73 V_rec: 6.65 V_kld: 811.88
No. 5 T_rec: 4.08 T_kld: 971.60 V_rec: 6.66 V_kld: 827.37
No. 6 T_rec: 4.07 T_kld: 1018.72 V_rec: 6.62 V_kld: 907.50
No. 7 T_rec: 4.02 T_kld: 1045.78 V_rec: 6.64 V_kld: 921.13
No. 8 T_rec: 4.01 T_kld: 1070.71 V_rec: 6.62 V_kld: 950.76
No. 9 T_rec: 3.95 T_kld: 1095.63 V_rec: 6.60 V_kld: 983.89
No. 10 T_rec: 3.99 T_kld: 1107.47 V_rec: 6.61 V_kld: 994.33
No. 11 T_rec: 3.93 T_kld: 1109.64 V_rec: 6.66 V_kld: 979.75
No. 12 T_rec: 3.95 T_kld: 1130.15 V_rec: 6.59 V_kld: 1019.33
No. 13 T_rec: 3.90 T_kld: 1132.29 V_rec: 6.60 V_kld: 1031.79
No. 14 T_rec: 3.91 T_kld: 1138.37 V_rec: 6.55 V_kld: 1052.47
No. 15 T_rec: 3.92 T_kld: 1134.55 V_rec: 6.72 V_kld: 1012.96
No. 16 T_rec: 3.92 T_kld: 1145.89 V_rec: 6.68 

100%|██████████| 5/5 [00:00<00:00, 44.16it/s]


No. 51 T_rec: 3.85 T_kld: 0.00 V_rec: 6.19 V_kld: 0.00
No. 52 T_rec: 3.85 T_kld: 0.00 V_rec: 6.18 V_kld: 0.00
No. 53 T_rec: 3.87 T_kld: 0.00 V_rec: 6.16 V_kld: 0.00
No. 54 T_rec: 3.83 T_kld: 0.00 V_rec: 6.15 V_kld: 0.00
No. 55 T_rec: 3.86 T_kld: 0.00 V_rec: 6.13 V_kld: 0.00
No. 56 T_rec: 3.83 T_kld: 0.00 V_rec: 6.11 V_kld: 0.00
No. 57 T_rec: 3.84 T_kld: 0.00 V_rec: 6.10 V_kld: 0.00
No. 58 T_rec: 3.80 T_kld: 0.00 V_rec: 6.08 V_kld: 0.00
No. 59 T_rec: 3.82 T_kld: 0.00 V_rec: 6.06 V_kld: 0.00
No. 60 T_rec: 3.84 T_kld: 0.00 V_rec: 6.04 V_kld: 0.00
save model 60...


100%|██████████| 5/5 [00:00<00:00, 65.80it/s]


No. 61 T_rec: 3.81 T_kld: 0.00 V_rec: 6.03 V_kld: 0.00
No. 62 T_rec: 3.78 T_kld: 0.00 V_rec: 6.01 V_kld: 0.00
No. 63 T_rec: 3.79 T_kld: 0.00 V_rec: 5.99 V_kld: 0.00
No. 64 T_rec: 3.78 T_kld: 0.00 V_rec: 5.97 V_kld: 0.00
No. 65 T_rec: 3.80 T_kld: 0.00 V_rec: 5.95 V_kld: 0.00
No. 66 T_rec: 3.79 T_kld: 0.00 V_rec: 5.94 V_kld: 0.00
No. 67 T_rec: 3.74 T_kld: 0.00 V_rec: 5.92 V_kld: 0.00
No. 68 T_rec: 3.77 T_kld: 0.00 V_rec: 5.91 V_kld: 0.00
No. 69 T_rec: 3.72 T_kld: 0.00 V_rec: 5.89 V_kld: 0.00
No. 70 T_rec: 3.75 T_kld: 0.00 V_rec: 5.88 V_kld: 0.00
save model 70...


...

No. 981 T_rec: 2.37 T_kld: 0.00 V_rec: 2.71 V_kld: 0.00
No. 982 T_rec: 2.38 T_kld: 0.00 V_rec: 2.71 V_kld: 0.00
No. 983 T_rec: 2.35 T_kld: 0.00 V_rec: 2.71 V_kld: 0.00
No. 984 T_rec: 2.34 T_kld: 0.00 V_rec: 2.71 V_kld: 0.00
No. 985 T_rec: 2.35 T_kld: 0.00 V_rec: 2.71 V_kld: 0.00
No. 986 T_rec: 2.36 T_kld: 0.00 V_rec: 2.71 V_kld: 0.00
No. 987 T_rec: 2.35 T_kld: 0.00 V_rec: 2.71 V_kld: 0.00
No. 988 T_rec: 2.38 T_kld: 0.00 V_rec: 2.70 V_kld: 0.00
No. 989 T_rec: 2.34 T_kld: 0.00 V_rec: 2.70 V_kld: 0.00
No. 990 T_rec: 2.34 T_kld: 0.00 V_rec: 2.70 V_kld: 0.00
save model 990...


100%|██████████| 5/5 [00:00<00:00, 39.10it/s]


No. 991 T_rec: 2.34 T_kld: 0.00 V_rec: 2.70 V_kld: 0.00
No. 992 T_rec: 2.35 T_kld: 0.00 V_rec: 2.70 V_kld: 0.00
No. 993 T_rec: 2.36 T_kld: 0.00 V_rec: 2.69 V_kld: 0.00
No. 994 T_rec: 2.34 T_kld: 0.00 V_rec: 2.69 V_kld: 0.00
No. 995 T_rec: 2.36 T_kld: 0.00 V_rec: 2.69 V_kld: 0.00
No. 996 T_rec: 2.34 T_kld: 0.00 V_rec: 2.69 V_kld: 0.00
No. 997 T_rec: 2.34 T_kld: 0.00 V_rec: 2.69 V_kld: 0.00
No. 998 T_rec: 2.32 T_kld: 0.00 V_rec: 2.69 V_kld: 0.00
No. 999 T_rec: 2.35 T_kld: 0.00 V_rec: 2.68 V_kld: 0.00


100%|██████████| 30000/30000 [09:12<00:00, 54.26it/s] 


: 